In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [21]:
sparql = SPARQLWrapper("https://landregistry.data.gov.uk/landregistry/query")
sparql.setReturnFormat(JSON)

In [28]:

sparql.setQuery("""

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
prefix ukhpi: <http://landregistry.data.gov.uk/def/ukhpi/>
prefix lrppi: <http://landregistry.data.gov.uk/def/ppi/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix lrcommon: <http://landregistry.data.gov.uk/def/common/>

# Returns house price index average prices for flats, semis, detacted and terraced for all LA since 2015, with GSS codes

SELECT ?regionName ?code ?date ?hpi ?hpiDetached ?hpiFlatMaisonette ?hpiSemiDetached ?hpiTerraced ?averagePriceDetached ?averagePriceFlatMaisonette ?averagePriceSemiDetached ?averagePriceTerraced
{
  BIND( now() AS ?currentDateTime ) .
  BIND( CONCAT( str(year(?currentDateTime)-6), "-", str(month(?currentDateTime)), "-", str(day(?currentDateTime)) ) AS ?currentDateString ) .
  
  ?region ukhpi:refPeriodStart ?date;
          ukhpi:housePriceIndex ?hpi;
          ukhpi:housePriceIndexFlatMaisonette ?hpiFlatMaisonette;
          ukhpi:averagePriceFlatMaisonette ?averagePriceFlatMaisonette.
          
  
  OPTIONAL{?region ukhpi:housePriceIndexDetached ?hpiDetached.}.
  OPTIONAL{?region ukhpi:housePriceIndexSemiDetached ?hpiSemiDetached.}.
  OPTIONAL{?region ukhpi:housePriceIndexTerraced ?hpiTerraced.}.
  OPTIONAL{?region ukhpi:averagePriceDetached ?averagePriceDetached.}.
  OPTIONAL{?region ukhpi:averagePriceSemiDetached ?averagePriceSemiDetached.}.
  OPTIONAL{?region ukhpi:averagePriceTerraced ?averagePriceTerraced.}.
          

  
  
  ?region ukhpi:refRegion ?regionRef.
  
  ?regionRef rdfs:seeAlso ?code.
             
  ?regionRef rdfs:label ?regionName.
  
  FILTER (langMatches( lang(?regionName), "EN")&&
         ?date > xsd:date(?currentDateString)).
             
  FILTER contains(str(?code),"gov").
  }
    """
)

try:
    ret = sparql.queryAndConvert()

#     for r in ret["results"]["bindings"]:
#         print(r)
except Exception as e:
    print(e)

pd.json_normalize(ret["results"]["bindings"])

,regionName.type,regionName.xml:lang,regionName.value,code.type,code.value,date.type,date.datatype,date.value,hpi.type,hpi.datatype,...,averagePriceDetached.value,averagePriceFlatMaisonette.type,averagePriceFlatMaisonette.datatype,averagePriceFlatMaisonette.value,averagePriceSemiDetached.type,averagePriceSemiDetached.datatype,averagePriceSemiDetached.value,averagePriceTerraced.type,averagePriceTerraced.datatype,averagePriceTerraced.value
0,literal,en,City of Aberdeen,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-11-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,348402,literal,http://www.w3.org/2001/XMLSchema#integer,97637,literal,http://www.w3.org/2001/XMLSchema#integer,199249,literal,http://www.w3.org/2001/XMLSchema#integer,142863
1,literal,en,City of Aberdeen,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2020-09-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,311630,literal,http://www.w3.org/2001/XMLSchema#integer,100211,literal,http://www.w3.org/2001/XMLSchema#integer,184682,literal,http://www.w3.org/2001/XMLSchema#integer,133110
2,literal,en,City of Aberdeen,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-02-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,343245,literal,http://www.w3.org/2001/XMLSchema#integer,100435,literal,http://www.w3.org/2001/XMLSchema#integer,197783,literal,http://www.w3.org/2001/XMLSchema#integer,140990
3,literal,en,City of Aberdeen,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2020-10-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,309470,literal,http://www.w3.org/2001/XMLSchema#integer,100633,literal,http://www.w3.org/2001/XMLSchema#integer,185160,literal,http://www.w3.org/2001/XMLSchema#integer,133860
4,literal,en,City of Aberdeen,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-10-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,353353,literal,http://www.w3.org/2001/XMLSchema#integer,100868,literal,http://www.w3.org/2001/XMLSchema#integer,203402,literal,http://www.w3.org/2001/XMLSchema#integer,146036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28284,literal,en,Hastings,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-11-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,559975,literal,http://www.w3.org/2001/XMLSchema#integer,175535,literal,http://www.w3.org/2001/XMLSchema#integer,386691,literal,http://www.w3.org/2001/XMLSchema#integer,299413
28285,literal,en,Orkney Islands,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-06-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,308488,literal,http://www.w3.org/2001/XMLSchema#integer,132002,literal,http://www.w3.org/2001/XMLSchema#integer,198326,literal,http://www.w3.org/2001/XMLSchema#integer,176435
28286,literal,en,Hastings,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-08-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,560192,literal,http://www.w3.org/2001/XMLSchema#integer,178196,literal,http://www.w3.org/2001/XMLSchema#integer,388744,literal,http://www.w3.org/2001/XMLSchema#integer,300826
28287,literal,en,Hastings,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#date,2022-09-01,literal,http://www.w3.org/2001/XMLSchema#decimal,...,564566,literal,http://www.w3.org/2001/XMLSchema#integer,178306,literal,http://www.w3.org/2001/XMLSchema#integer,391454,literal,http://www.w3.org/2001/XMLSchema#integer,302685


In [29]:
df=pd.json_normalize(ret["results"]["bindings"])

In [30]:
df['code']=df['code.value'].str.split('/',expand=True)[5]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28289 entries, 0 to 28288
Data columns (total 36 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   regionName.type                      28289 non-null  object
 1   regionName.xml:lang                  28289 non-null  object
 2   regionName.value                     28289 non-null  object
 3   code.type                            28289 non-null  object
 4   code.value                           28289 non-null  object
 5   date.type                            28289 non-null  object
 6   date.datatype                        28289 non-null  object
 7   date.value                           28289 non-null  object
 8   hpi.type                             28289 non-null  object
 9   hpi.datatype                         28289 non-null  object
 10  hpi.value                            28289 non-null  object
 11  hpiDetached.type                     2820

In [32]:
df2=df[['regionName.value', 'code','date.value','hpi.value','hpiDetached.value','hpiFlatMaisonette.value','hpiSemiDetached.value','hpiTerraced.value', 'averagePriceDetached.value','averagePriceFlatMaisonette.value','averagePriceSemiDetached.value','averagePriceTerraced.value' ]]

In [33]:
df2

,regionName.value,code,date.value,hpi.value,hpiDetached.value,hpiFlatMaisonette.value,hpiSemiDetached.value,hpiTerraced.value,averagePriceDetached.value,averagePriceFlatMaisonette.value,averagePriceSemiDetached.value,averagePriceTerraced.value
0,City of Aberdeen,S12000033,2022-11-01,72.02,79.64,65.90,77.29,76.86,348402,97637,199249,142863
1,City of Aberdeen,S12000033,2020-09-01,69.68,71.23,67.64,71.64,71.61,311630,100211,184682,133110
2,City of Aberdeen,S12000033,2022-02-01,72.45,78.46,67.79,76.72,75.85,343245,100435,197783,140990
3,City of Aberdeen,S12000033,2020-10-01,69.88,70.74,67.93,71.83,72.01,309470,100633,185160,133860
4,City of Aberdeen,S12000033,2022-10-01,73.87,80.77,68.08,78.90,78.56,353353,100868,203402,146036
...,...,...,...,...,...,...,...,...,...,...,...,...
28284,Hastings,E07000062,2022-11-01,188.25,198.60,171.59,198.94,195.33,559975,175535,386691,299413
28285,Orkney Islands,S12000023,2022-06-01,189.48,193.29,174.13,189.80,185.50,308488,132002,198326,176435
28286,Hastings,E07000062,2022-08-01,189.60,198.67,174.19,199.99,196.25,560192,178196,388744,300826
28287,Hastings,E07000062,2022-09-01,190.54,200.22,174.30,201.39,197.46,564566,178306,391454,302685


In [34]:
df2.to_csv('landreg.csv',index=False)